In [ ]:
%pip install tensorflow numpy
%pip install scipy

In [1]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import pickle
import numpy as np
import os 
import argparse

from OriginalGridNetwork.ratSimulator import RatSimulator
from OriginalGridNetwork.dataGenerator import dataGenerator
from OriginalGridNetwork.agent import Network
from OriginalGridNetwork.trainer import Trainer
from OriginalGridNetwork.showGridCells import showGridCells
from OriginalGridNetwork.helper import prepare_testing_data, train_agent, test_agent


In [2]:
LSTM_units = 128
linear_layer_units = 512
pc_units = 256
hc_units = 12

num_steps = 800
num_features = 3
num_trajectories = 500


learning_rate=1e-4
clipping=1e-5
weightDecay=1e-5
batch_size = 10
SGDSteps = 300000
global_step = 0

In [3]:
random_state = np.random.RandomState(seed = 10)
pc_centers = random_state.uniform(0, 2.2, size = (pc_units, 2))
hc_centers = random_state.uniform(-np.pi, np.pi, size = (hc_units))

In [4]:
tf.compat.v1.reset_default_graph()
sess = tf.compat.v1.Session()
agent = Network(sess, lr=learning_rate, hu=LSTM_units, lu=linear_layer_units, place_units=pc_units, head_units=hc_units, clipping=clipping, weightDecay=weightDecay, batch_size=batch_size, num_features=num_features, n_steps=num_steps)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


I0000 00:00:1749440490.253773  359963 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled


In [ ]:
if os.path.exists("agentBackup"):
    print("Loading saved model...")
    agent.save_restore_Model(restore=True)
    global_step = sess.run(agent.epoch)
    print(f"Loaded model at step {global_step}")
else:
    global_step = 0
    print("No saved model found. Starting from scratch.")

In [ ]:
try:
    trainer, global_step = train_agent(agent, pc_units, hc_units, pc_centers, hc_centers, num_steps,
                                   num_features, SGDSteps, num_trajectories, batch_size, global_step)
    print(f"Training finished. Saving model at step {global_step}")
    agent.save_restore_Model(restore=False, epoch=global_step)
except (KeyboardInterrupt, SystemError, SystemExit, MemoryError, RuntimeError) as e:
    print(f"Training interrupted ({str(e)}). Saving model at step {global_step}")
    agent.save_restore_Model(restore=False, epoch=global_step)
    raise

In [ ]:
input_data_test, pos_test, init_LSTMState_test = prepare_testing_data(pc_units, hc_units, pc_centers, hc_centers)
test_agent(trainer, agent, pc_centers, input_data_test, init_LSTMState_test, pos_test, global_step)


In [ ]:
bins = 32
showGridCells(agent, num_trajectories, num_steps, num_features, pc_units, hc_units, linear_layer_units, bins, pc_centers, hc_centers)